In [1]:
import torch
import torch.functional as F
import torch.nn as nn
import spacy
import pandas as pd
import numpy as np
import random
import time
from torch.utils.data import Dataset, DataLoader
from spacy.lang.en.stop_words import STOP_WORDS
SEED=42
torch.manual_seed(SEED)
import re

In [2]:
df = pd.read_excel('Participants_Data_News_category/Data_Train.xlsx')
df = df.sample(frac = 1, random_state=42, axis = 0)
df.SECTION = df.SECTION.astype(int)

In [10]:
# min_count = np.min(df.SECTION.value_counts())
# min_count
# df2 = pd.DataFrame(columns=['STORY', 'SECTION'])
# texts = []
# classes = []
# for cat in range(4):
#     temp_idxs = df.index[df.SECTION==cat][:min_count]
#     frames = [df.iloc[temp_idxs, :], df2]
#     df2 = pd.concat(frames, sort=False)

# len(df2) / min_count
# df2.reset_index(drop = True, inplace=True)
# df2.reset_index(drop = True, inplace=True)

In [13]:
class Vocabulary(object):
    """Class to build vocabulary for mapping"""
    def __init__(self, token_to_idx = None, add_unk = True, unk_token = "<UNK>", mask_token="<MASK>", begin_seq_token="<BEGIN>",end_seq_token="<END>"):
        """
        Args:
            token_to_idx: Initialize token to idx dictionary.
            add_unk: Whether to include the unknown token in the vocabulary
            unk_token: How the unknown token is represented in the vocabulary
        """
        
        if token_to_idx is None:
            token_to_idx = {}
            
        self._token_to_idx = token_to_idx
        
        self._idx_to_token = {idx:token 
                             for token, idx in self._token_to_idx.items()}
        
#         self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

#         self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)
        
        self._add_unk = add_unk
        
    def from_serializable(self):
        """
        Returns:
            A dictionary that can be serialized
        """

        return cls(**contents)
        
    def add_token(self, token):
        """ Update mapping dictionaries given the token
        Args:
            token (str): Token to add to the vocabulary
        Returns:
            index (int): The index corresponding to the token          
        """

        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token

        return index
        
    def lookup_token(self, token):
        """ Retrieves the index associated with the token
            or the UNK index if the token isn't present in the vocabulary

        Args:
            token (str): The token for which the index has to be retrieved
        Returns:
            index (int): The index associated with the token in the dictionary

        Note: 
            'UNK Index' has to be >=0 for the UNK functionality
        """

        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx.get(token)

    def lookup_index(self, index):
        """Retrieve the token associated with the index
        Args:
            index (int): The index to look up
        Returns:
            token (str): The token associated with the index
        Raises:
            KeyError: if the index is not in the vocabulary
        """

        if index not in self._idx_to_token:
            raise KeyError("the index %d is not in the vocabulary" % index)
        else:
            return self._idx_to_token[index]

    def __len__(self):
        """Returns the length of the vocabulary
        """
        return len(self._token_to_idx)      


In [14]:
class News_Category(Dataset):
    def __init__(self, df, nlp, vocab):
        """Initializing
        Args:
            df (Pandas DataFrame): Dataframe consisting of tweets and labels
            nlp (spacy object): For preprocessing
            vocab (Vocabulary Object): To vectorize the tweets
        """
        self.df = df
        self.nlp = nlp
        self.vocab = vocab
        measure_len = lambda context: len(context.split(" "))
        self._max_seq_length = max(map(measure_len, self.df.STORY)) + 1
        
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
          
        tweet =  self.df.STORY.iloc[idx]
        
        label = self.df.SECTION.iloc[idx]
        return {'tweet':torch.LongTensor(self.preprocess(tweet)), 'label':label}
    
    def preprocess(self, sent):
        
        #Preprocessing and tokenizing
        text = sent
        text = text.lower()
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "can not ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r"\'scuse", " excuse ", text)
        text = re.sub('\W', ' ', text)
        text = re.sub('\s+', ' ', text)
        text = text.strip(' ')
        text = re.sub('[^a-zA-Z]',' ',text)
        text=text.lower()
        sent = text
        sent =  " ".join(sent.split())
        sent = [token.lemma_ for token in self.nlp(sent) if token.text not in STOP_WORDS]
        sent = self.vectorize(sent)
        return sent
    
    def vectorize(self, sent):
        """Converts raw text to numeric vectors using the vocabulary
        Args:
            sent (str): The tweet to be vectorized
        Returns:
            vector (list): The vector associated with the tweet
        """
        vector = [self.vocab.begin_seq_index]
#         vector = []
        for token in sent:
            vector.append(vocab.lookup_token(token))
        vector.append(self.vocab.end_seq_index)
            
        return vector
    

In [15]:
vocab = Vocabulary(token_to_idx=None, add_unk=True)
nlp = spacy.load(name='en_core_web_sm')

def clean(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\\'s", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    text = re.sub('[^a-zA-Z]',' ',text)
    text=text.lower()
    text = " ".join(text.split())
    text = [token.lemma_ for token in nlp(text) if token.text not in STOP_WORDS]
    text = [w for w in text if len(w)>1]
    return text



In [25]:
cleaned_tweets = df.STORY.apply(clean)
text = []
for i in range(len(cleaned_tweets)):
    for word in cleaned_tweets[i]:
        text.append(word)
len(text)
from collections import Counter
count_dict = Counter(text).most_common(len(set(text))-500)
for tup in count_dict:
    vocab.add_token(tup[0])

In [40]:
data = News_Category(df,nlp, vocab)

In [43]:
def load_glove(filepath):
    """Loads the glove embeddings
    
    Args:
        filepath (str): path to the glove embeddings file
    Return:
        word_to_index (dict): Mappings from word to index
        embeddings (np.array): Embeddings of the words in the vocabulary
    """
    word_to_index = {}
    embeddings = []
    with open(filepath, 'r', encoding='utf-8') as fp:
        for index, line in enumerate(fp):
            line = line.split(" ")
            word_to_index[line[0]] = index
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    
    return word_to_index, np.array(embeddings)

In [44]:
def make_embedding_matrix(filepath, words):
    """Create embedding matrix for a specific set of words
    Args:
        word_to_index (dict) : mapping of word to index
        embeddings (list): embeddings of words
        words (list): List of words in the dictionary
    Returns:
        final_embeddings (np..array) : embedding matrix
    """
    
    word_to_idx, embeddings = load_glove(filepath)
    embedding_size = embeddings.shape[1]
    final_embeddings = np.zeros((len(words), embedding_size))
    
    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.zeros(embedding_size)
            final_embeddings[i, :] = embedding_i
            
    return final_embeddings

In [45]:
words=[]
for idx in range(0, vocab.__len__()):
    words.append(vocab.lookup_index(idx))
embs = make_embedding_matrix(r"C:\Users\win10\Documents\glove.6B\glove.6B.300d.txt", words)

In [48]:
i = random.randint(0,4200)
print(i)
data[i]['tweet'], data[i]['label']

3944


(tensor([    1, 21507,  4780, 10259, 22374, 16099, 13212, 10903, 10034, 10188,
          1128, 23584,  6083, 17023, 22395,  5363, 14999, 22649,   831,  2676,
         22649,  7010, 11023,  6185,  6810, 22738,  6083,     0, 12490,  3440,
         17486,  7766, 14999,  3392, 10754, 18956, 12846,  8133, 22738,  1128,
         23584,  2519, 13212,  4847,     2]),
 0)

In [49]:
# class LSTMClassifier(nn.Module):
#     def __init__(self, embedding_dim, hidden_dim, label_size, batch_size, embedding_weights, num_layers = 1,bidirectional = False):
#         super(LSTMClassifier, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.batch_size = batch_size
#         self.embedding_dim = embedding_dim
#         self.word_embeddings = nn.Embedding.from_pretrained(embedding_weights, freeze=False, padding_idx=0)
#         self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim,
#                             num_layers = num_layers, bidirectional = bidirectional,batch_first=True)
#         if bidirectional:
#             self.fc = nn.Linear(hidden_dim*2, label_size)
#         else:
#             self.fc = nn.Linear(hidden_dim, label_size)
 
#     def forward(self, sentences, train = True):
#         embeds = self.word_embeddings(sentences)
#         packed_outputs, (hidden,cell) = self.lstm(embeds)
#         dense_outputs = self.fc(hidden[1])
#         outputs = dense_outputs
#         return outputs

In [50]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size, embedding_weights, num_layers = 1,bidirectional = False):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        self.attention = nn.MultiheadAttention(embed_dim=self.embedding_dim, num_heads=1)
        self.word_embeddings = nn.Embedding.from_pretrained(embedding_weights, freeze=False, padding_idx=0)
        self.lstm = nn.LSTM(input_size = self.embedding_dim, hidden_size = hidden_dim,
                            num_layers = num_layers, bidirectional = bidirectional,batch_first=True)
        if bidirectional:
            self.fc = nn.Linear(hidden_dim*2, label_size)
        else:
            self.fc = nn.Linear(hidden_dim, label_size)
 
    def forward(self, sentences, train = True):
        embeds = self.word_embeddings(sentences)
        embeds = embeds.permute(1,0,2)
        attn_output, _ = self.attention(embeds, embeds, embeds)
        packed_outputs, (hidden,cell) = self.lstm(attn_output.permute(1,0,2))
        dense_outputs = self.fc(hidden[1])
        outputs = dense_outputs
        return outputs

In [51]:
# test_nn = nn.LSTM(input_size = 100, hidden_size = 100,
#                 num_layers = 2, bidirectional = False,batch_first=True)
# print(test_nn(torch.rand(32,8,100))[1][0].shape)

In [52]:
BATCH_SIZE = 64
nlabel = 4
num_layers = 2
hidden_dim = 512
EMBEDDING_DIM = embs.shape[1]
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = LSTMClassifier(embedding_dim=EMBEDDING_DIM,hidden_dim=hidden_dim,label_size=nlabel, batch_size=BATCH_SIZE, embedding_weights=torch.from_numpy(embs).float(), num_layers=num_layers)
model = model.to(device)
 
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)
loss_function = nn.CrossEntropyLoss()
 
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [53]:
#Rewrite collate_ FN function, whose input is the sample data of a batch
def collate_fn(batch):
	#Because token_ List is a variable length data, so you need to use a list to load the token of the batch_ list
    token_lists = [item['tweet'] for item in batch]
    #Each label is an int. we take out all the labels in the batch and reassemble them
    labels = [item['label'] for item in batch]
    #Converting labels to tensor
    labels = torch.LongTensor(labels)
    return {
    'token_list': torch.nn.utils.rnn.pad_sequence(token_lists, batch_first=True),
    'label': labels,
    }

#When using dataloader to load data, pay attention to collate_ The FN parameter passes in an overridden function
trainset = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [54]:
for batch in trainset:
    print(batch['token_list'])
    print(batch['label'])
    break

tensor([[    1, 21327, 15011,  ...,     0,     0,     0],
        [    1,  9455,  4973,  ...,     0,     0,     0],
        [    1, 21214, 13289,  ...,     0,     0,     0],
        ...,
        [    1, 23685,   441,  ...,     0,     0,     0],
        [    1,   159, 10999,  ...,     0,     0,     0],
        [    1, 19429,  1598,  ...,     0,     0,     0]])
tensor([1, 0, 2, 1, 2, 3, 3, 2, 2, 3, 1, 0, 2, 0, 0, 0, 1, 2, 2, 2, 3, 1, 2, 1,
        1, 1, 1, 0, 1, 2, 2, 2, 0, 3, 2, 2, 1, 1, 3, 1, 0, 2, 3, 1, 2, 0, 1, 0,
        0, 2, 2, 1, 1, 2, 1, 2, 0, 2, 0, 2, 2, 1, 1, 1])


In [55]:
from tqdm import tqdm
epochs=25
for epoch in range(epochs):
    time.sleep(1)
    total_loss = 0.0
    total_acc=0.0
    for i, batch in enumerate(tqdm(trainset)):
        feature, label = batch['token_list'].to(device), batch['label'].to(device)
#         batch_length = torch.tensor(33, dtype = torch.int64).unsqueeze(0)
        optimizer.zero_grad()
        output =  model(feature).squeeze()
        loss = loss_function(output, label)
        acc=categorical_accuracy(output,label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_acc += acc.item() 
        
    scheduler.step()    
    print(f"loss on epoch {epoch+1} = {total_loss/len(trainset)}")
    print(f"accuracy on epoch {epoch+1} = {total_acc/len(trainset)}")

100%|██████████| 120/120 [02:40<00:00,  1.34s/it]


loss on epoch 1 = 0.47069135028868914
accuracy on epoch 1 = 0.8333767364422481


100%|██████████| 120/120 [02:49<00:00,  1.42s/it]


loss on epoch 2 = 0.12602893096239617
accuracy on epoch 2 = 0.9631076390544574


100%|██████████| 120/120 [02:47<00:00,  1.40s/it]


...

loss on epoch 23 = 0.004865084525939286
accuracy on epoch 23 = 0.9975260416666667


100%|██████████| 120/120 [02:30<00:00,  1.26s/it]


loss on epoch 24 = 0.004625556626729121
accuracy on epoch 24 = 0.9975260416666667


100%|██████████| 120/120 [02:30<00:00,  1.25s/it]

loss on epoch 25 = 0.004508763731723775
accuracy on epoch 25 = 0.9973958333333334


In [ ]:
sample = pd.read_excel('Participants_Data_News_category/Sample_submission.xlsx')
test_set = pd.read_excel('Participants_Data_News_category\Data_Test.xlsx')

In [57]:
def predict_class(model, text, min_len = 4):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    text = re.sub('[^a-zA-Z]',' ',text)
    text=text.lower()
    text = " ".join(text.split())
    text = [token.lemma_ for token in nlp(text) if token.text not in STOP_WORDS]
    vector = []
    for token in text:
        vector.append(vocab.lookup_token(token))
    tensor = torch.LongTensor(vector)
    tensor = tensor.unsqueeze(0)
    preds = model(tensor.to(device))
    pred_class = preds.argmax(dim = 1)
    return pred_class[0]
#     print('The sentence is : {}'.format(sent))
    # print(f'Predicted class is: {pred_class.item()} = {itol[int(pred_class)]}.')

In [58]:
model.eval()
preds = []
for idx in tqdm(test_set.index):
    preds.append(predict_class(model,test_set.loc[idx, 'STORY']).item())
    

100%|██████████| 2748/2748 [01:01<00:00, 44.82it/s]


In [59]:
submission = pd.DataFrame(preds, columns=['SECTION'])
submission.to_csv('submission.csv')